# Inicio
[Enunciado.](https://docs.google.com/document/d/1ZfZAJFjaKTtWtREfNAPUXhxnf6J42qI-mAg6vF1uWUA/) |
[Informe.](https://docs.google.com/document/d/1a9VOFq1fccPuiFBY21JuMnR78QpHkrwxKfeiG6ZmFiw/) |
[Repositorio.](https://github.com/NicoDeGiacomo/ODD-TP2)

In [164]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [165]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [166]:
# TRAIN SET
url = "https://raw.githubusercontent.com/NicoDeGiacomo/ODD-TP2/main/train_set.csv"
train_df = pd.read_csv(url)
train_df.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,Billing_Country,Account_Name,Opportunity_Name,Opportunity_ID,Sales_Contract_No,Account_Owner,Opportunity_Owner,Account_Type,Opportunity_Type,Quote_Type,Delivery_Terms,Opportunity_Created_Date,Brand,Product_Type,Size,Product_Category_B,Price,Currency,Last_Activity,Quote_Expiry_Date,Last_Modified_Date,Last_Modified_By,Product_Family,Product_Name,ASP_Currency,ASP,ASP_(converted)_Currency,ASP_(converted),Planned_Delivery_Start_Date,Planned_Delivery_End_Date,Month,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A
0,27761,EMEA,None,1,1,1,1,0,Bureaucratic_Code_4,6/16/2015,None,Netherlands,Account_Name_619,Opportunity_Name_12598,0,None,Person_Name_51,Person_Name_18,Account_Type_2,Opportunity_Type_1,Non Binding,Delivery_Terms_2,12/7/2015,None,None,None,None,None,None,NaT,1/30/2016,6/13/2016,Person_Name_18,Product_Family_77,Product_Name_99,EUR,0.52,USD,0.58817,5/1/2016,6/30/2016,2016 - 5,Q2,2016,NaT,10,EUR,5272800.0,EUR,5272800.0,Closed Lost,Prod_Category_A_None
1,27760,EMEA,None,0,0,0,0,0,Bureaucratic_Code_4,6/16/2015,None,Netherlands,Account_Name_619,Opportunity_Name_12600,1,39,Person_Name_51,Person_Name_20,Account_Type_2,Opportunity_Type_1,Non Binding,Delivery_Terms_2,12/7/2015,None,None,None,None,None,None,NaT,1/20/2016,1/15/2016,Person_Name_20,Product_Family_77,Product_Name_100,EUR,0.53,USD,0.59948,1/18/2016,1/20/2016,2016 - 1,Q1,2016,NaT,0,EUR,48230.0,EUR,48230.0,Closed Won,Prod_Category_A_None
2,27446,Americas,NW America,0,0,0,0,0,Bureaucratic_Code_4,4/21/2015,Source_7,United States,Account_Name_1794,Opportunity_Name_469,2,760,Person_Name_64,Person_Name_8,Account_Type_5,Opportunity_Type_1,Non Binding,Delivery_Terms_4,12/8/2015,None,None,None,None,None,None,NaT,9/30/2016,9/29/2016,Person_Name_8,Product_Family_81,Product_Name_91,USD,0.48,USD,0.48000,1/25/2016,1/25/2016,2016 - 1,Q1,2016,NaT,0,USD,83865.6,USD,83865.6,Closed Won,Prod_Category_A_None
3,16808,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,Source_11,United States,Account_Name_1201,Opportunity_Name_415,3,None,Person_Name_8,Person_Name_8,Account_Type_5,Opportunity_Type_19,Non Binding,Delivery_Terms_1,12/8/2015,Other,Product_Type_0,Size_4,Product_Category_B_16,0.42,USD,NaT,NaT,3/27/2018,Person_Name_8,Product_Family_209,Product_Name_432,USD,0.53,USD,0.53000,2/1/2018,3/31/2018,2018 - 2,Q1,2018,NaT,14,USD,7421881.5,USD,7421881.5,Closed Lost,Prod_Category_A_None
4,16805,Americas,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,Source_11,United States,Account_Name_1201,Opportunity_Name_851,4,None,Person_Name_8,Person_Name_8,Account_Type_5,Opportunity_Type_19,Non Binding,Delivery_Terms_1,12/8/2015,Other,Product_Type_0,Size_4,Product_Category_B_16,0.42,USD,NaT,6/2/2017,3/27/2018,Person_Name_8,Product_Family_209,Product_Name_432,USD,0.53,USD,0.53000,2/1/2018,2/28/2018,2018 - 2,Q1,2018,NaT,25,USD,13357192.5,USD,13357192.5,Closed Lost,Prod_Category_A_None


In [167]:
# TEST SET
url = "https://raw.githubusercontent.com/NicoDeGiacomo/ODD-TP2/main/test_set.csv"
test_df = pd.read_csv(url)
test_df.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,Billing_Country,Account_Name,Opportunity_Name,Opportunity_ID,Sales_Contract_No,Account_Owner,Opportunity_Owner,Account_Type,Opportunity_Type,Quote_Type,Delivery_Terms,Opportunity_Created_Date,Brand,Product_Type,Size,Product_Category_B,Price,Currency,Last_Activity,Quote_Expiry_Date,Last_Modified_Date,Last_Modified_By,Product_Family,Product_Name,ASP_Currency,ASP,ASP_(converted)_Currency,ASP_(converted),Planned_Delivery_Start_Date,Planned_Delivery_End_Date,Month,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A
0,6140,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_171,Product_Name_332,EUR,0.325,USD,0.36761,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,162240.0,EUR,367419.0,Prod_Category_A_None
1,6146,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_167,Product_Name_315,EUR,0.320,USD,0.36195,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,78624.0,EUR,367419.0,Prod_Category_A_None
2,6151,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_140,Product_Name_241,EUR,0.275,USD,0.31105,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,126555.0,EUR,367419.0,Prod_Category_A_None
3,6118,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6117,10690,6504,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/25/2019,Person_Name_13,Product_Family_171,Product_Name_328,EUR,0.325,USD,0.36761,6/1/2019,6/28/2019,2019 - 6,Q2,2019,NaT,1,EUR,243360.0,EUR,757783.5,Prod_Category_A_None
4,6124,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6117,10690,6504,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/25/2019,Person_Name_13,Product_Family_167,Product_Name_315,EUR,0.320,USD,0.36195,6/1/2019,6/29/2019,2019 - 6,Q2,2019,NaT,0,EUR,157248.0,EUR,757783.5,Prod_Category_A_None


# Feature Engineering

## Limpieza de datos

In [168]:
# Limpieza de datos general, donde eliminamos columnas que no deben ser tomadas en cuenta.

# Nos quedamos solamente con los estados cerrados
train_df = train_df[(train_df["Stage"] == "Closed Won")|(train_df["Stage"] == "Closed Lost")]

# Asumimos que se establecen luego de confirmada la compra
# Planned_Delivery_Start_Date, Planned_Delivery_End_Date, Delivery_Quarter, Delivery_Year, Actual_Delivery_Date, Month
train_df.drop(["Planned_Delivery_Start_Date", "Planned_Delivery_End_Date", "Delivery_Quarter", "Delivery_Year", "Actual_Delivery_Date", "Month"], axis=1, inplace=True, errors="ignore")
test_df.drop(["Planned_Delivery_Start_Date", "Planned_Delivery_End_Date", "Delivery_Quarter", "Delivery_Year", "Actual_Delivery_Date", "Month"], axis=1, inplace=True, errors="ignore")

# Solo 14 rows tienen un valor distinto de cero
#len(train_df[train_df["Submitted_for_Approval"] == 0])
train_df.drop(["Submitted_for_Approval"], axis=1, inplace=True, errors="ignore")
test_df.drop(["Submitted_for_Approval"], axis=1, inplace=True, errors="ignore")

# Tienen todos valores NaT
# Last_Activity Prod_Category_A
# train_df["Last_Activity"].value_counts()
train_df.drop(["Prod_Category_A", "Last_Activity"], axis=1, inplace=True, errors="ignore")
test_df.drop(["Prod_Category_A", "Last_Activity"], axis=1, inplace=True, errors="ignore")

# Tienen cerca de 16k de valores en None
# train_df["Price"].value_counts()
train_df.drop(["Brand", "Product_Type", "Size", "Product_Category_B", "Price", "Currency", "ASP_(converted)_Currency", "Actual_Delivery_Date", "ASP_(converted)"], axis=1, inplace=True, errors="ignore")
test_df.drop(["Brand", "Product_Type", "Size", "Product_Category_B", "Price", "Currency", "ASP_(converted)_Currency", "Actual_Delivery_Date", "ASP_(converted)"], axis=1, inplace=True, errors="ignore")

# Account_Created es irrelevante
train_df.drop(["Account_Created_Date"], axis=1, inplace=True, errors="ignore")
test_df.drop(["Account_Created_Date"], axis=1, inplace=True, errors="ignore")

# Quote_Expiry_Date parece irrelevante
train_df.drop(["Quote_Expiry_Date"], axis=1, inplace=True, errors="ignore")
test_df.drop(["Quote_Expiry_Date"], axis=1, inplace=True, errors="ignore")

# Opportunity_Name no aporta
train_df.drop(["Opportunity_Name"], axis=1, inplace=True, errors="ignore")
test_df.drop(["Opportunity_Name"], axis=1, inplace=True, errors="ignore")

# Sales_Contract_No, Last_Modified_By y Last_Modified_Date no es información disponible al momento de realizar la predicción.
# No debería usarse, para evitar target leakage.
train_df.drop(["Sales_Contract_No", "Last_Modified_By", "Last_Modified_Date"], axis=1, inplace=True, errors="ignore")
test_df.drop(["Sales_Contract_No", "Last_Modified_By", "Last_Modified_Date"], axis=1, inplace=True, errors="ignore")

# La columna ID no debería influir en el destino de la oportunidad
train_df.drop(["ID"], axis=1, inplace=True, errors="ignore")
test_df.drop(["ID"], axis=1, inplace=True, errors="ignore")

# Eliminarmos el espacio de la columna Source
train_df.rename(columns={'Source ':'Source'}, inplace=True, errors="ignore")
test_df.rename(columns={'Source ':'Source'}, inplace=True, errors="ignore")

## Modificacion y creacion de features

In [169]:
!pip install currencyconverter
from currency_converter import CurrencyConverter
from datetime import date, datetime

In [170]:
# Hay que unificar las monedas de los campos:
# Total_Amount (Total_Amount_Currency)
# ASP (ASP_Currency)
# Total_Taxable_Amount (Total_Taxable_Amount_Currency)
# Hay 5 monedas distintas en el dataset: "USD", "JPY", "EUR", "AUD", "GBP"
# La conversión se debe realizar con el tipo de cambio de la fecha de la oportunidad

c = CurrencyConverter(fallback_on_missing_rate=True)

def convert_to_usd(args):
    amount, currency, s_date = args
    return c.convert(amount, currency, 'USD', date=datetime.strptime(s_date, '%m/%d/%Y').date())

train_df=train_df.assign(**{'Total_Amount_USD': train_df[["Total_Amount", "Total_Amount_Currency", "Opportunity_Created_Date"]].apply(convert_to_usd, axis=1)})
train_df=train_df.assign(**{'ASP_USD': train_df[["ASP", "ASP_Currency", "Opportunity_Created_Date"]].apply(convert_to_usd, axis=1)})
train_df=train_df.assign(**{'Total_Taxable_Amount_USD': train_df[["Total_Taxable_Amount", "Total_Taxable_Amount_Currency", "Opportunity_Created_Date"]].apply(convert_to_usd, axis=1)})

test_df=test_df.assign(**{'Total_Amount_USD': test_df[["Total_Amount", "Total_Amount_Currency", "Opportunity_Created_Date"]].apply(convert_to_usd, axis=1)})
test_df=test_df.assign(**{'ASP_USD': test_df[["ASP", "ASP_Currency", "Opportunity_Created_Date"]].apply(convert_to_usd, axis=1)})
test_df=test_df.assign(**{'Total_Taxable_Amount_USD': test_df[["Total_Taxable_Amount", "Total_Taxable_Amount_Currency", "Opportunity_Created_Date"]].apply(convert_to_usd, axis=1)})

# Ahora puedo eliminar las columnas anteriores
train_df.drop(["Total_Amount", "Total_Amount_Currency","ASP", "ASP_Currency","Total_Taxable_Amount", "Total_Taxable_Amount_Currency"], axis=1, inplace=True, errors="ignore")
test_df.drop(["Total_Amount", "Total_Amount_Currency","ASP", "ASP_Currency","Total_Taxable_Amount", "Total_Taxable_Amount_Currency"], axis=1, inplace=True, errors="ignore")

In [171]:
# Separamos Opportunity_Created_Date en distintas columnas
train_df=train_df.assign(**{'Opportunity_Created_Month': train_df[["Opportunity_Created_Date"]].apply(lambda x: datetime.strptime(x[0], '%m/%d/%Y').month, axis=1)})
train_df=train_df.assign(**{'Opportunity_Created_Day': train_df[["Opportunity_Created_Date"]].apply(lambda x: datetime.strptime(x[0], '%m/%d/%Y').day, axis=1)})
train_df=train_df.assign(**{'Opportunity_Created_Year': train_df[["Opportunity_Created_Date"]].apply(lambda x: datetime.strptime(x[0], '%m/%d/%Y').year, axis=1)})

# Separamos Opportunity_Created_Date en distintas columnas
test_df=test_df.assign(**{'Opportunity_Created_Month': test_df[["Opportunity_Created_Date"]].apply(lambda x: datetime.strptime(x[0], '%m/%d/%Y').month, axis=1)})
test_df=test_df.assign(**{'Opportunity_Created_Day': test_df[["Opportunity_Created_Date"]].apply(lambda x: datetime.strptime(x[0], '%m/%d/%Y').day, axis=1)})
test_df=test_df.assign(**{'Opportunity_Created_Year': test_df[["Opportunity_Created_Date"]].apply(lambda x: datetime.strptime(x[0], '%m/%d/%Y').year, axis=1)})

# Ahora puedo eliminar las columnas anteriores
train_df.drop(["Opportunity_Created_Date"], axis=1, inplace=True, errors="ignore")
test_df.drop(["Opportunity_Created_Date"], axis=1, inplace=True, errors="ignore")

In [172]:
# Las columnas "Pricing, Delivery_Terms_Quote_Appr" y "Pricing, Delivery_Terms_Approved" se pueden resumir en una.
# Las columnas "Bureaucratic_Code_0_Approval" y "Bureaucratic_Code_0_Approved" se pueden resumir en una.
def is_approved(args):
    requested, approved = args
    if requested == approved:
      return 1
    if requested == 1:
      return 0
    return 1

train_df=train_df.assign(**{'Pricing_And_Delivery_OK': train_df[["Pricing, Delivery_Terms_Quote_Appr", "Pricing, Delivery_Terms_Approved"]].apply(is_approved, axis=1)})
test_df=test_df.assign(**{'Pricing_And_Delivery_OK': test_df[["Pricing, Delivery_Terms_Quote_Appr", "Pricing, Delivery_Terms_Approved"]].apply(is_approved, axis=1)})

train_df=train_df.assign(**{'Bureaucratic_OK': train_df[["Bureaucratic_Code_0_Approval", "Bureaucratic_Code_0_Approved"]].apply(is_approved, axis=1)})
test_df=test_df.assign(**{'Bureaucratic_OK': test_df[["Bureaucratic_Code_0_Approval", "Bureaucratic_Code_0_Approved"]].apply(is_approved, axis=1)})


# Ahora puedo eliminar las columnas anteriores
train_df.drop(["Pricing, Delivery_Terms_Quote_Appr", "Pricing, Delivery_Terms_Approved","Bureaucratic_Code_0_Approval", "Bureaucratic_Code_0_Approved"], axis=1, inplace=True, errors="ignore")
test_df.drop(["Pricing, Delivery_Terms_Quote_Appr", "Pricing, Delivery_Terms_Approved","Bureaucratic_Code_0_Approval", "Bureaucratic_Code_0_Approved"], axis=1, inplace=True, errors="ignore")

## Encondings

### Common Init

In [173]:
# Aplicamos un ecoding a las variables categoricas
# Posibles: OneHotEncoding, LabelEncoding, FrequencyEncoding

# Copiamos el dataframe para aplicar encodings varios
train_df_enc = train_df.copy()
test_df_enc = test_df.copy()

# Convertimos el Stage en 1/0 (aplica a todos los encodings) (solo al set de entrenamiento)
train_df_enc["Stage"]=train_df_enc["Stage"].replace({'Closed Won':1 , 'Closed Lost':0})

### One Hot Encoding

In [ ]:
# Aplicamos OneHotEncoding
train_df_enc_onehot = pd.get_dummies(train_df_enc, columns = ["Region", "Territory", "Bureaucratic_Code", "Source", "Billing_Country", "Account_Name", "Account_Owner", "Opportunity_Owner", "Account_Type", "Opportunity_Type","Quote_Type","Delivery_Terms","Product_Family","Product_Name"], dummy_na=False, drop_first=True)
test_df_enc_onehot = pd.get_dummies(test_df_enc, columns = ["Region", "Territory", "Bureaucratic_Code", "Source", "Billing_Country", "Account_Name", "Account_Owner", "Opportunity_Owner", "Account_Type", "Opportunity_Type","Quote_Type","Delivery_Terms","Product_Family","Product_Name"], dummy_na=False, drop_first=True)

### Frequency Enconding

In [202]:
# Aplicamos FrequencyEncoding
attributes_to_encode = ["Region", "Territory", "Bureaucratic_Code", "Source", "Billing_Country", "Account_Name", "Account_Owner", "Opportunity_Owner", "Account_Type", "Opportunity_Type","Quote_Type","Delivery_Terms","Product_Family","Product_Name"]
train_df_enc_freq = train_df_enc.copy()
test_df_enc_freq = test_df_enc.copy()

for attr in attributes_to_encode:
  freq_encoder = (train_df_enc.groupby(attr).size()) / len(train_df_enc)
  train_df_enc_freq['enc_'+attr] = train_df_enc[attr].apply(lambda x : freq_encoder[x])

  freq_encoder = (test_df_enc.groupby(attr).size()) / len(test_df_enc)
  test_df_enc_freq['enc_'+attr] = test_df_enc[attr].apply(lambda x : freq_encoder[x])

# Elimino las columnas encodeadas
train_df_enc_freq.drop(attributes_to_encode, axis=1, inplace=True, errors="ignore")
test_df_enc_freq.drop(attributes_to_encode, axis=1, inplace=True, errors="ignore")

### Common End

In [203]:
# Agrupamos por Opportunity_ID los diferentes sets
# Revisar si MAX es lo mejor a usar
train_set = train_df.groupby("Opportunity_ID").max()
test_set = test_df.groupby("Opportunity_ID").max()

train_df_enc_onehot = train_df_enc_onehot.groupby("Opportunity_ID").max()
test_df_enc_onehot = test_df_enc_onehot.groupby("Opportunity_ID").max()

train_df_enc_freq = train_df_enc_freq.groupby("Opportunity_ID").max()
test_df_enc_freq = test_df_enc_freq.groupby("Opportunity_ID").max()

# Modelos

## Modelo 1
 * XGBoost

In [204]:
# Separamos la variable a predecir (Stage)
xg_boost_train_y = train_df_enc_freq["Stage"]
xg_boost_train_x = train_df_enc_freq.drop(["Stage"], axis=1)

xg_boost_test_x = test_df_enc_freq.copy()

In [205]:
import xgboost as xgb

# Generamos el modelo
xg_boost = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

# Entrenamos el modelo
xg_boost.fit(xg_boost_train_x,xg_boost_train_y)

# Predecimos
#xg_preds = xg_boost.predict(xg_boost_test_x)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

## Modelo 2
Usamos Distributed Random Forest (H2ORandomForestEstimator)

In [ ]:

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

!pip install h2o
import h2o
from h2o.estimators import H2ORandomForestEstimator

#h2o.shutdown()
h2o.init()

In [ ]:
h2o_train=h2o.H2OFrame(train_set)
h2o_test=h2o.H2OFrame(test_set)

h2o_train["Stage"] = h2o_train["Stage"].asfactor()
predictors = ['Region', 'Territory', 'Bureaucratic_Code', 'Source', 'Billing_Country', 'Account_Name', 'Opportunity_Name', 'Account_Owner', 'Opportunity_Owner', 'Account_Type', 'Opportunity_Type', 'Quote_Type', 'Delivery_Terms', 'Product_Family', 'Product_Name', 'TRF', 'Stage', 'Total_Amount_USD', 'ASP_USD', 'Total_Taxable_Amount_USD', 'Opportunity_Created_Month', 'Opportunity_Created_Day', 'Opportunity_Created_Year', 'Pricing_And_Delivery_OK', 'Bureaucratic_OK']
response = "Stage"

train, valid = h2o_train.split_frame(ratios=[.8], seed=1234)

drf = h2o.estimators.H2ORandomForestEstimator(nfolds=2, ntrees=50, max_depth=5, min_rows=5, binomial_double_trees=True, calibrate_model=True,calibration_frame=valid)
drf.train(x=predictors, y=response, training_frame=train, validation_frame=valid)

# Eval performance:
perf = drf.model_performance()

# Generate predictions on a set:
pred = drf.predict(h2o_test)

perf

In [ ]:
pred

In [ ]:
# Le damos el formato correcto a las predicciones
pred["Opportunity_ID"] = h2o.H2OFrame(test_set.reset_index()[["Opportunity_ID"]])
pred["Target"] = pred["cal_Closed Won"]
# Las exportamos a Drive
#h2o.export_file(pred[["Opportunity_ID", "Target"]], "/content/drive/MyDrive/Datos/pred7_cal.csv", force=True)

## Modelo 3

In [ ]:
# Modelo 1